<a href="https://colab.research.google.com/github/falcowinkler/tweet-classification/blob/master/TwitterFakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import re
nltk.download('stopwords')
ps = PorterStemmer() 
def tokenize(line):
    # TODO can we use named entity recognition for location, as an indicator / feature?
    """Takes a tweet and turns it into whitespace seperated string of tokens"""
    line = re.sub("[^a-zA-Z]", ' ', line) # replace non digits
    # replace all urls with keyword, and throw rest away?
    words = line.split()
    words = [word for word in words if not word in set(stopwords.words('english'))] # remove articles?
    words = [ps.stem(word) for word in words]
    line = ' '.join(words)
    # TODO: word 2 vec?
    return line
    
def get_dataset(df):
    df.text=df.text.map(tokenize)
    print(df.text)
    tokenizer = Tokenizer(split=' ', num_words=750)
    tokenizer.fit_on_texts(df.text)
    encoded_urls = tokenizer.texts_to_matrix(df.text, mode='count')
    labels = np.array(df.target)
    return encoded_urls, labels

df = pd.read_csv('train.csv', sep=',')
features, labels = get_dataset(df)

encoded_words_train , encoded_words_test , labels_train , labels_test = \
train_test_split(features,labels,test_size=0.20, random_state=42, shuffle =True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0           our deed reason earthquak may allah forgiv us
1                    forest fire near La rong sask canada
2       all resid ask shelter place notifi offic No ev...
3             peopl receiv wildfir evacu order california
4       just got sent photo rubi alaska smoke wildfir ...
                              ...                        
7608    two giant crane hold bridg collaps nearbi home...
7609    aria ahrari thetawniest the control wild fire ...
7610         M utc km S volcano hawaii http co zdtoyd ebj
7611    polic investig e bike collid car littl portug ...
7612    the latest more home raze northern california ...
Name: text, Length: 7613, dtype: object


In [1]:
?Tokenizer

Object `Tokenizer` not found.


In [28]:
lg = LogisticRegression(max_iter=200)
lg.fit(encoded_words_train,labels_train)
print(lg.score(encoded_words_test, labels_test))

0.7793827971109653


In [29]:
vocab_size = 750
embedding_dim = 1
batch_size= 64
rnn_units = 50

# TODO: how to build GRU model?
model = keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.summary()
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])
model.fit(encoded_words_train, labels_train, epochs=100, batch_size=batch_size)

print('Done training!')
model.evaluate(encoded_words_test, labels_test)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 1)             750       
_________________________________________________________________
gru_4 (GRU)                  (64, None, 50)            7950      
_________________________________________________________________
dense_6 (Dense)              (64, None, 1)             51        
Total params: 8,751
Trainable params: 8,751
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
95/96 [============================>.] - ETA: 0s - loss: 0.2460 - accuracy: 0.5691

InvalidArgumentError: ignored